In [7]:
import urllib, json
import pandas as pd

In [8]:
def get_table(league_code, gw_start, gw_end):
    league_url = 'https://fantasy.premierleague.com/api/leagues-classic/%d/standings/' % league_code
    response = urllib.request.urlopen(league_url)
    data = json.loads(response.read())
    teams = {}
    for i in range(len(data.get('standings').get('results'))):
        team_id = data.get('standings').get('results')[i].get('entry')
        name = data.get('standings').get('results')[i].get('player_name')
        teams[team_id] = name

    df = pd.DataFrame(columns=['id', 'name', 'points'])
    for team_id in teams.keys():
        total = 0
        team_url = 'https://fantasy.premierleague.com/api/entry/%d/history/' % team_id
        response = urllib.request.urlopen(team_url)
        data = json.loads(response.read())
        for gw_no in range(gw_start, gw_end+1):
            try:
                total += data.get('current')[gw_no].get('points')
                total -= data.get('current')[gw_no].get('event_transfers_cost')
            except IndexError:
                total += 0
        d = pd.DataFrame({'id':team_id, 'name':teams[team_id], 'points':total}, index=[0])
        df = df.append(d)
    df = df.sort_values(['points'], ascending=False)
    return df

In [10]:
get_table(399, 0, 45)

,id,name,points
0,354,Ashwin Sapre,2231
0,17392,Vibhav D,2183
0,4002,Dungeon Master,2167
0,2427841,Harsh gupta,2099
0,566326,Pranav Naranatt,2097
0,324744,Rajat Kulkarni,2054
0,834254,Chinmay More,2015
0,409440,uday malhotra,2008
0,47606,Aniruddh Naik,1987
0,713394,Ojas Atkar,1950
